In [102]:
def fast_pow(c, N, d):
    d_bin = "{0:b}".format(d)
    d_len = len(d_bin)
    reductions = 0
    h = 0
    x = c

    for j in range(1, d_len):
        x, r = mod_reduce(x ** 2, N)
        reductions = reductions + r
        if d_bin[j] == "1":
            x, r = mod_reduce(x * c, N)
            reductions = reductions + r
            h = h + 1

    return x, h, reductions

def mod_reduce(a, b):
    reductions = 0
    if a >= b:
        a = a % b
        reductions = 1
    return a, reductions


In [103]:
from sympy import randprime, mod_inverse
import math
import numpy as np
import random

def GenModulus(w):
    n = len(w) // 2
    p = randprime(2 ** n, 2 ** (n+1))
    q = randprime(2 ** n, 2 ** (n+1))
    N = p * q
    return N, p, q

def GenRSA(w):
    N, p, q = GenModulus(w)
    m = (p-1) * (q-1)
    e = 2 ** 16 + 1
    d = mod_inverse(e, m)
    return N, e, d, p, q

def enc(x, N, e):
    return fast_pow(x, N, e) #x ** e % N

def dec(c, N, d):
    return fast_pow(c, N, d) #c ** d % N

## Task 1

In [104]:
w = '1' * 70
N, e, d, p, q = GenRSA(w)

print(N, e, d, p, q)

2474851085089393729489 65537 941120874317071099009 50589345497 48920401337


In [105]:
d_bin = "{0:b}".format(d)
d_bin

'1100110000010010101101000000001110101101010110000100011010010010000001'

In [106]:
def generate_nums(bits):
    exponent = int(bits + '1', 2)

    first_end = min(100000, math.ceil(N ** (1 / exponent)))
    first = range(2, first_end)

    second_start = math.ceil(N ** (1 / exponent)) + 1
    second_end = min(math.ceil(N ** (1 / (exponent - 1))), second_start + 100000)
    second = range(second_start, second_end)

    return list(first), list(second)

In [107]:
def guess_bits():
    bits = '1'
    
    while True: 
        nums = generate_nums(bits)
        

        if min(len(nums[0]), len(nums[1])) <= 1:
            return bits

        print(f"{nums[0][0]} - {nums[0][-1]} ; {nums[1][0]} - {nums[1][-1]}")

        reductions = []
        for first_num, second_num in zip(*nums):
            reductions.append((
                dec(first_num, N, d)[2],
                dec(second_num, N, d)[2]
            ))

        v = np.var(reductions)
        print(f"Var = {v}")
        bits += '0' if v  <= 1 else '1'

In [108]:
guess_bits()

2 - 99999 ; 13526426 - 13626425
Var = 0.29702410344532815
2 - 18998 ; 19000 - 118999
Var = 0.1318083877283256
2 - 238 ; 240 - 472
Var = 2.0740297297795136


'1001'

## Task 2 (Blind signatures)

In [153]:
import hashlib
import struct
import random
import sympy

def rand_coprime(x):
    y =  random.randint(2, x)
    while math.gcd(r, N) != 1:
        y = random.randint(2, x)
    
    return y

def fastpow(x, N, e):
    return fast_pow(x, N, e)[0]

# def h(num):
#     return int(hashlib.sha256(str(num).encode('ASCII')).hexdigest()[:5], 16)

def sign(m, N, d):
    return fastpow(m, N, d)

def verify(m, s, N, e):
    return m == fastpow(s, N, e)


def blind_sign(m, N, d):
    r = rand_coprime(N)
    r_inv = mod_inverse(r, N)
    m_prime = m * fast_pow(r, N, e)[0] % N
    return sign(m_prime, N, d) * r_inv % N


In [154]:
N, e, d, p, q = GenRSA("1" * 100)
N, e, d

(3280063719696351696207369088433, 65537, 1134461515393703107137208493393)

In [156]:
m = 100

usual = sign(m, N, d)
blind = blind_sign(m, N, d)

print(f"Usual: {usual}\nBlind: {blind}")

Usual: 1917899468749281425228222038248
Blind: 1917899468749281425228222038248


In [158]:
verify(m, usual, N, e), verify(m, blind, N, e)

(True, True)

## Task 3

In [161]:
def blind_dec(c, N, e, d):
    r = rand_coprime(N)
    r_inv = mod_inverse(r, N)
    m_prime, h, r = dec(c * fast_pow(r, N, e)[0] % N, N, d)
    return m_prime * r_inv % N, h, r

In [166]:
c = 1000

blind_dec(c, N, e, d), dec(c, N, d)

((319855790163060887118454472407, 45, 144),
 (319855790163060887118454472407, 45, 140))